## Convolutions

In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
torch.set_printoptions(linewidth=120)

Um Netzwerk zu verbessern, möchte man gerne weitere Layers anfügen (das Netzwerk soll tiefer werden). Um zu verhindern, dass die Anzahl der zu trainierenden Parameter zu stark ansteigt, fasst man über die Konvolution-Operation benachbarte Werten zu einem Wert zusammen. 

<img src="./img/convolution01.png" width="600">

In [5]:
x = torch.tensor([
    [3,0,1,2],
    [1,5,8,9],
    [2,7,2,5],
    [0,1,3,1]
])

filter = torch.tensor([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]])

x = x.reshape(1,1,4,4)
filter = filter.reshape(1, 1, 3, 3)
F.conv2d(x, filter)


tensor([[[[ -5,  -4],
          [-10,  -2]]]])

#### Edge detection

Wir stellen uns die Bilderkennung so vor, dass die Filter in frühen Layers eines CNN einfache Features erkenne, spätere Layer erkennen kompliziertere Strukturen.

<img src="./img/convolution02.png" width="700">

Das Beispiel zeigt, wie sich mit einem Filter vertikale Linien erkennen lassen.

<img src="./img/convolution03.png" width="700">

Das Resultat der Convolution-Operation hebt die vertikalen Linie hervor. 

<img src="./img/convolution04.png" width="500">



In [32]:
x = torch.tensor([
    [10,10,10,0,0,0],
    [10,10,10,0,0,0], 
    [10,10,10,0,0,0], 
    [10,10,10,0,0,0],
    [10,10,10,0,0,0],
    [10,10,10,0,0,0]
]) 

filter = torch.tensor([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]])

x = x.reshape(1,1,6,6)
filter = filter.reshape(1, 1, 3, 3)
F.conv2d(x, filter)


tensor([[[[ 0, 30, 30,  0],
          [ 0, 30, 30,  0],
          [ 0, 30, 30,  0],
          [ 0, 30, 30,  0]]]])

Weitere Beispiele

In [30]:
x = torch.tensor([
    [10,10,10,10,10,10],
    [10,10,10,10,10,10], 
    [10,10,10,10,10,10], 
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0]
]) 

filter = torch.tensor([[1, 1, 1],
                       [0, 0, 0],
                       [-1, -1, -1]])

x = x.reshape(1,1,6,6)
filter = filter.reshape(1, 1, 3, 3)
F.conv2d(x, filter)

tensor([[[[ 0,  0,  0,  0],
          [30, 30, 30, 30],
          [30, 30, 30, 30],
          [ 0,  0,  0,  0]]]])

In [33]:
x = torch.tensor([
    [10,10,10,0,0,0],
    [10,10,10,0,0,0], 
    [10,10,10,0,0,0], 
    [0,0,0,10,10,10],
    [0,0,0,10,10,10],
    [0,0,0,10,10,10]
]) 

filter = torch.tensor([[1, 1, 1],
                       [0, 0, 0],
                       [-1, -1, -1]])

x = x.reshape(1,1,6,6)
filter = filter.reshape(1, 1, 3, 3)
F.conv2d(x, filter)

tensor([[[[  0,   0,   0,   0],
          [ 30,  10, -10, -30],
          [ 30,  10, -10, -30],
          [  0,   0,   0,   0]]]])

Man kann auch Varianten probieren:

In [ ]:
sobel_filter = torch.tensor([[1, 0, -1],
                             [2, 0, -2],
                             [1, 0, -1]])

sobel_filter = torch.tensor([[3, 0, -3],
                             [10, 0, -10],
                             [3, 0, -3]])

Häufig lässt man das Netzwerk selbst geeignete Werte der Filter finden. Die Werte der Filter sind dann zu trainierende Parameter. Filter werden auch manchmal *kernel* genannt. *kernel_size* ist der Parameter, mit dem wir in Pytorch die Größe eines zu trainierenden Filters definieren.




#### Padding

In den bisherigen Beispielen war die Ausgabe der Convolusion eine kleinere Matrix als die Ausgangsmatrix. 

```
Input: n
Filter: f
Output: n-f+1
```

Nachteile:
- das Bild wird immer kleiner
- die Randpixel gehen weniger häufig in die Ausgabe ein.

Um die Nachteile zu verhindern, wird das Ursprungsbild mit einem Rand versehen, üblicherweise mit 0-Werten (Padding)

In [4]:
# Hier ist das Padding manuell eingefügt
x = torch.tensor([
    [0,0,0,0,0,0],
    [0,3,0,1,2,0],
    [0,1,5,8,9,0],
    [0,2,7,2,5,0],
    [0,0,1,3,1,0],
    [0,0,0,0,0,0]
]) 

filter = torch.tensor([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]])

x = x.reshape(1,1,6,6)
filter = filter.reshape(1, 1, 3, 3)
F.conv2d(x, filter)


tensor([[[[ -5,  -5,  -6,   9],
          [-12,  -5,  -4,  11],
          [-13, -10,  -2,  13],
          [ -8,  -3,   2,   5]]]])

Der Funktion *conv2d* können wir dazu einen *padding*-Parameter mitgeben.

In [13]:
# Hier wird das Padding über einen Parameter gesteuert.
x = torch.tensor([
    [3,0,1,2],
    [1,5,8,9],
    [2,7,2,5],
    [0,1,3,1]
])

filter = torch.tensor([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]])

x = x.reshape(1,1,4,4)
filter = filter.reshape(1, 1, 3, 3)
F.conv2d(x, filter,padding=(1,1))     # Anzahl Zeros auf beiden Seiten jeder Dimension

tensor([[[[ -5,  -5,  -6,   9],
          [-12,  -5,  -4,  11],
          [-13, -10,  -2,  13],
          [ -8,  -3,   2,   5]]]])

```
Input: n
Filter: f
Padding: p
Output: n+2p-f+1

```

Eine Konvolusion ohne Padding heißt *valid*-Konvolusion. <br>
Eine Konvolusion mit gleicher Größe von Input und Output heißt *same*-Konvolusion.  

Für eine *same*-Konvolusion gilt:

$$p = \frac{f-1}{2}$$

f ist so gut wie immer ungerade.

#### Stride

Mit *striding* steuern wir, wieviel Schritte der Filter bis zum nächsten Halt macht.

In [18]:
x = torch.tensor([
    [3,0,1,2,8],
    [1,5,8,9,4],
    [2,7,2,5,3],
    [0,1,3,1,2],
    [4,2,9,3,2],
])

filter = torch.tensor([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]])

x = x.reshape(1,1,5,5)
filter = filter.reshape(1, 1, 3, 3)
F.conv2d(x, filter,stride=2)

tensor([[[[-5, -4],
          [-8,  7]]]])

```
Input: n
Filter: f
Padding: p
Stride: s
Output: (n+2p-f)/s + 1

```

#### Volume-Convolutions

Für ein Farbbild benötigen wir einen 3-dimensionalen Filter. Die Konvolusion-Operation addiert die Werte, die der Filter-Würfel liefert, zu einer Zahl.

In [26]:
x = torch.tensor([
    [[3,0,1],
     [1,5,8],
     [2,7,2]],
     
    [[0,1,2],
     [5,2,5],
     [0,1,3]],
    
    [[2,0,1],
     [2,2,3],
     [2,1,1]]
])

filter = torch.tensor([
    [[1, 2],
     [1, 0]],
    
    [[0, 1],
     [1, 0]],
    
    [[3, 1],
     [0, 2]]
])
      

x = x.reshape(1,3,3,3)
filter = filter.reshape(1, 3, 2, 2)
F.conv2d(x, filter)

tensor([[[[20, 18],
          [25, 45]]]])

Beispiel: Ein 6x6x3 Input mit einem 3x3x3 Filter ergibt einen 4x4x1 Output:

<img src="./img/convolution05.png" width="700">

#### Mehrere Filter

Um mehrere Features parallel zu entdecken, wendet man mehrere Filter gleichzeitig an und stapelt die jeweiligen Outputs hintereinander.

<img src="./img/convolution06.png" width="600">

Beispiel: Wenn ein 32x32x3 Bild mit 10 3x3x3 Filtern über Konvolution verknüpft wird (ohne padding und stride), dann entsteht daraus ein
30x30x10 Output.

#### Layer

Ein Layer eines CNN besteht darin, den Output der Konvolution mit einem Bias zu Versehen, durch eine Aktivierungsfunktion zu schicken und die Resultate hintereinander zu stapeln.

<img src="./img/convolution07.png" width="900">


Übung: Wieviele Parameter hat ein CNN-Layer, der aus 10 3x3x3 Filtern besteht?  Ist die Anzahl der Parameter abhängig von der Größe des Inputs? (Antwort: 280, nein).

In [17]:
# Übung: was wird ausgegeben?

inputs = torch.randn(5,3,50,50)
filters = torch.randn(8,3,4,4)
output = F.conv2d(inputs, filters, padding = (1,1), stride = 2)
output.shape



 

torch.Size([5, 8, 25, 25])

#### Ein CNN-Layer für das Fashion-MNIST dataset

In [27]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        
     
    def forward(self, t):
        t = self.conv1(t)
        t = F.relu(t)
        return t

In [28]:
net = Net()
inputs = torch.randn(20,1,28,28)   # Das Format eines Fashion-MNIST Batches der Größe 20
output = net(inputs)
output.shape

torch.Size([20, 6, 24, 24])

In [34]:
# die Anzahl der zu trainierenden Parameter
for x in list(net.conv1.parameters()):
    print(x.numel())

150
6


Übung
```
CNN-Layer 1: f=3,s=1,p=0, anzFilter = 10 
CNN-Layer 2: f=5,s=2,p=0, anzFilter = 20
CNN-Layer 3: f=5,s=2,p=0, anzFilter = 40

Input: 20x3x39x39
Welche Dimension hat der Output?
Antwort: 20x40x7x7
```

In [37]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=2)
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=5, stride=2)
     
    def forward(self, t):
        t = self.conv1(t)
        t = self.conv2(t)
        t = self.conv3(t)
     
        return t

net = Net()
input = torch.randn(20,3,39,39)
output = net(input)
output.shape

torch.Size([20, 40, 7, 7])

Die Kunst beim Design von CNNs ist es, die richtigen Hyperparameter zu wählen: wieviele Filter, welche Größe sollen sie haben, Werte für geeignete Wert für padding und stride finden.

#### Pooling

In CNNs gibt es neben den Convolution-Layers üblicherweise auch Pooling-Layers. Am häufigsten findet man *max-pooling*

In [44]:
x = torch.tensor([
    [3,0,1,2],
    [1,2,8,9],
    [2,1,2,4],
    [0,1,3,1]
]).float()

x = x.reshape(1,1,4,4)
output = F.max_pool2d(x, kernel_size=2, stride=2)
print(output)

tensor([[[[3., 9.],
          [2., 4.]]]])


Max-Pooling hat sich in vielen Fällen als vorteilhaft erweisen. Versuch einer Erklärung: Wenn eine hohe Zahl bedeutet, dass ein Feature entdeckt wurde, dann sorgt max-Pooling dafür, dass diese Entdeckung bestehen bleibt (und nicht durch umliegende kleinere Zahlen wieder in Frage gestellt wird (sort of)).

Max-Pooling Layers führen keine zusätzliche zu trainierenden Parameter ein (sie werden ausschließlich durch ihre Hyperparameter festgelegt).

#### Vorteile von CNN-Layers

CNN-Layers haben deutlich weniger Parameter als fully-connected-Layers

<img src="./img/convolution08.png" width="400">

Fully-connected-Layer: ca. 14 Millionen Parameter, 
CNN-Layer: 156 Parameter

Wie kommt die Einsparung von Parametern zustande?

**Parameter sharing**:
Wir stellen uns einen Filter als einen feature-detector vor(z.B. vertikale Kanten). Wenn ein solcher feature-detector in einem Teil des Bildes nützlich ist, dann vermutlich auch in einem anderen Teil des Bildes. Aus der Menge der Input-Signale können also unterschiedliche Bereiche dieselben Paramter benutzen, um features zu entdecken. Deswegen ist es sinnvoll, sich mit demselben Filter über den gesamten Input zu bewegen.

**Sparse connections**: Ein Output-Wert hängt nur noch von einem Teil der Input-Werte ab. 